In [1]:
import sys
import os

import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from fpgen.utils import pdb_to_uni_mol
from fpgen.prop_prediction.dataset import FPbase

In [3]:
fpbase_json = pd.read_json('../data/fpbase.json')
fpbase_json.head()

,agg,doi,genbank,ipg_id,name,pdb,seq,slug,states,switch_type,transitions,uniprot,uuid
0,,10.1126/science.273.5280.1392,None,None,10B,[],MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,10b,"[{'brightness': None, 'em_max': 525, 'ex_max':...",,[],None,RO9XQ
1,,10.1126/science.273.5280.1392,None,None,11,[],MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,11,"[{'brightness': None, 'em_max': 512, 'ex_max':...",b,[],None,W92N1
2,t,10.1126/science.1102506,None,None,22G,[2Z6X],MSVIKPDMKIKLRMEGAVNGHPFAIEGVGLGKPFEGKQSMDLKVKE...,22g,[],,[],None,1UZQ4
3,,10.1002/cbic.200300818,None,None,(3-F)Tyr-EGFP,[1RRX],SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,3-ftyr-egfp,"[{'brightness': None, 'em_max': 514, 'ex_max':...",b,[],None,2VPWT
4,,10.1126/science.273.5280.1392,None,None,5B,[],MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,5b,"[{'brightness': None, 'em_max': 524, 'ex_max':...",,[],None,G9FG2


In [4]:
fpbase_csv = pd.read_csv('../data/fpbase.csv')
fpbase_csv.head()

,sequence,brightness,em_max,ex_max,ext_coeff,lifetime,maturation,pka,stokes_shift,qy,agg,switch_type
0,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,NaN,525.0,513.0,30800.0,NaN,NaN,NaN,12.0,NaN,NaN,NaN
1,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,NaN,512.0,502.0,33000.0,NaN,NaN,NaN,10.0,NaN,NaN,b
2,MSVIKPDMKIKLRMEGAVNGHPFAIEGVGLGKPFEGKQSMDLKVKE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,NaN
3,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,NaN,514.0,484.0,NaN,NaN,NaN,NaN,30.0,NaN,NaN,b
4,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,NaN,524.0,512.0,19400.0,NaN,NaN,NaN,12.0,NaN,NaN,NaN


In [15]:
fpbase_csv.iloc[0]['sequence']

'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLGYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSYQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

In [16]:
seq = 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLGYGVQCFARYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSYQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'

In [22]:
def preproc(seq: str) -> dict:
    uuid = fpbase_json[fpbase_json['seq'] == seq]['uuid'].values[0]
    return pdb_to_uni_mol(f'../data/pdb_structures/{uuid}.pdb')

In [23]:
data = FPbase(preprocess_function=preproc)

In [31]:
x_train, y_train = data.get_train('em_max', is_scaled=True)
x_train = x_train.to_list()

In [ ]:
from unimol_tools import MolTrain, MolPredict

clf = MolTrain(
    task='classification', 
    data_type='molecule', 
    epochs=10, 
    batch_size=16, 
    metrics='auc',
    model_name='unimolv1', # avaliable: unimolv1, unimolv2
    model_size='84m', # work when model_name is unimolv2. avaliable: 84m, 164m, 310m, 570m, 1.1B.
)

pred = clf.fit(data = data)